In [81]:
import pandas as pd
dataset=pd.read_excel('YEHIA2.xlsx')
import arabicstopwords.arabicstopwords as asw
from nltk import word_tokenize
import string
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold, LeaveOneOut, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [82]:
def remve_noise(cases):
    corpus=[]
    for case in cases:
        s=case.lower()
        words=word_tokenize(s)
        word=[w for w in words if w not in asw.stopwords_list() if w not in string.punctuation]
        s=" ".join(word)
        corpus.append(s)
    return corpus

In [83]:
tfidf_vectorizer = TfidfVectorizer()
# tfidf_vectorizer = CountVectorizer()
tfidf_vectorizer.fit(remve_noise(dataset['text']))
X = tfidf_vectorizer.transform(remve_noise(dataset['text']))
Y=dataset['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2)

In [84]:
kfold = None
score = 0
for i in range(5, 100):
    k_folds = KFold(n_splits = i)
    scores = cross_val_score(classfier, X, Y, cv = k_folds)
    scoreTemp = scores.mean()
    if scoreTemp > score:
        kfold = i
        score = scoreTemp

In [85]:
print('kfold: ', kfold)
print('score: ', score)

kfold:  95
score:  0.7041148325358851


In [86]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
rbf = svm.SVC(kernel='rbf').fit(X_train, Y_train)
sig = svm.SVC(kernel='sigmoid').fit(X_train, Y_train)
poly = svm.SVC(kernel='poly').fit(X_train, Y_train)
classfier=MultinomialNB()
classfier.fit(X_train,Y_train)

MultinomialNB()

In [87]:
Y_pred=classfier.predict(X_test)
sig_pred = sig.predict(X_test)
rbf_pred = rbf.predict(X_test)
poly_pred=poly.predict(X_test)

In [88]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [89]:
print(confusion_matrix(Y_test,Y_pred))

[[141   0   0]
 [ 20   0   0]
 [ 38   0   0]]


In [90]:
print("accuracy is ",accuracy_score(Y_test,Y_pred))
print("accuracy is sig ",accuracy_score(Y_test, sig_pred))
print("accuracy is rbf ",accuracy_score(Y_test, rbf_pred))
print("accuracy is poly ",accuracy_score(Y_test, poly_pred))

accuracy is  0.7085427135678392
accuracy is sig  0.7135678391959799
accuracy is rbf  0.7085427135678392
accuracy is poly  0.7085427135678392


In [91]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

        اسره       0.71      1.00      0.83       141
         جنح       0.00      0.00      0.00        20
        مدني       0.00      0.00      0.00        38

    accuracy                           0.71       199
   macro avg       0.24      0.33      0.28       199
weighted avg       0.50      0.71      0.59       199



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [92]:
remve_noise(['السلام عليكم . سؤال منهما فضلكم   اشتريت  نصف شقة هل ممكن اسجل النصف الخاص بي فقط  . حيث ان الشريك الاخر  رافض التسجيل . وشكرا'])

['السلام سؤال فضلكم اشتريت نصف شقة ممكن اسجل النصف الخاص ان الشريك الاخر رافض التسجيل وشكرا']

In [93]:
set(dataset.label.values)

{'اسره', 'جنح', 'مدني'}

In [94]:
len(dataset)

992